In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install optuna==2.10.1
!pip install imblearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.2/308.2 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 KB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.1/147.1 KB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 KB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 KB 11.6 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=d8270f4942e82cadec7b8542b29577a97228c01b4c5db224e91f3322a5a6cedf
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip
Look

## 1. Tabular Data & Modules Load



In [8]:
import os, sys
from pathlib import Path

# Set up path
try:
    base_path = Path(__file__).resolve().parent.parent
except:
    %cd '/content/drive/MyDrive/Colab Notebooks/kaggle/Player-Contact-Detection/tutorial'
    base_path = Path('__file__').resolve().parent.parent

module_path = base_path / 'module'
data_path = base_path / 'data'

# Append system path 
sys.path.append(str(module_path))

/content/drive/MyDrive/Colab Notebooks/kaggle/Player-Contact-Detection/tutorial


In [9]:
# Import data analysis libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from xgboost.sklearn import XGBClassifier
from lightgbm.sklearn import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import matthews_corrcoef
from ensemble import BinaryCalssifier, Regressor
from jtlearn import ImbSampler

In [15]:
# Load datasets
tr_baseline_helmets = pd.read_csv(data_path / 'train_baseline_helmets.csv')
tr_labels = pd.read_csv(data_path / 'train_labels.csv')
tr_player_tracking = pd.read_csv(data_path / 'train_player_tracking.csv')
tr_video_metadata = pd.read_csv(data_path / 'train_video_metadata.csv')

te_baseline_helmets = pd.read_csv(data_path / 'test_baseline_helmets.csv')
te_player_tracking = pd.read_csv(data_path / 'test_player_tracking.csv')
te_video_metadata = pd.read_csv(data_path / 'test_video_metadata.csv')

submission = pd.read_csv(data_path / 'sample_submission.csv')

In [16]:
# 야드 to 미터 변환
yard2meter = 0.9144
convert_columns = ['x_position', 'y_position', 'speed', 'distance', 'acceleration', 'sa']
tr_player_tracking[convert_columns] = tr_player_tracking[convert_columns] * yard2meter
te_player_tracking[convert_columns] = te_player_tracking[convert_columns] * yard2meter
display(tr_player_tracking)
display(te_player_tracking)

,game_play,game_key,play_id,nfl_player_id,datetime,step,team,position,jersey_number,x_position,y_position,speed,distance,direction,orientation,acceleration,sa
0,58580_001136,58580,1136,44830,2021-10-10T21:08:20.900Z,-108,away,CB,22,56.317896,38.953440,1.014984,0.100584,320.33,263.93,0.649224,-0.585216
1,58580_001136,58580,1136,47800,2021-10-10T21:08:20.900Z,-108,away,DE,97,54.388512,24.515064,0.210312,0.009144,346.84,247.16,1.179576,0.822960
2,58580_001136,58580,1136,52444,2021-10-10T21:08:20.900Z,-108,away,FS,29,66.010536,28.767024,0.557784,0.054864,11.77,247.69,0.576072,-0.301752
3,58580_001136,58580,1136,46206,2021-10-10T21:08:20.900Z,-108,home,TE,86,52.459128,20.226528,0.338328,0.036576,127.85,63.63,0.630936,0.566928
4,58580_001136,58580,1136,52663,2021-10-10T21:08:20.900Z,-108,away,ILB,48,57.835800,25.146000,0.466344,0.045720,183.62,253.71,0.283464,0.283464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1353048,58575_003081,58575,3081,48476,2021-10-10T19:22:06.800Z,140,away,CB,40,66.092832,47.365920,1.453896,0.155448,345.36,342.68,0.448056,-0.374904
1353049,58575_003081,58575,3081,44887,2021-10-10T19:22:06.800Z,140,away,ILB,45,68.360544,49.834800,0.896112,0.091440,342.39,354.46,0.576072,-0.219456
1353050,58575_003081,58575,3081,44174,2021-10-10T19:22:06.800Z,140,away,ILB,49,68.159376,50.977800,1.024128,0.100584,352.79,349.20,0.576072,0.420624
1353051,58575_003081,58575,3081,45217,2021-10-10T19:22:06.800Z,140,away,TE,82,73.554336,44.595288,1.764792,0.182880,340.78,346.51,0.484632,-0.466344


,game_play,game_key,play_id,nfl_player_id,datetime,step,team,position,jersey_number,x_position,y_position,speed,distance,direction,orientation,acceleration,sa
0,58172_003247,58172,3247,41937,2020-09-13T19:30:20.200Z,-272,home,MLB,57,58.777632,10.323576,4.407408,0.420624,20.74,12.43,2.862072,2.743200
1,58172_003247,58172,3247,45345,2020-09-13T19:30:20.200Z,-272,away,ILB,50,69.521832,33.595056,2.880360,0.292608,179.52,184.91,1.709928,-1.655064
2,58172_003247,58172,3247,46205,2020-09-13T19:30:20.200Z,-272,home,DE,98,60.021216,14.392656,1.645920,0.182880,6.73,339.85,0.630936,-0.621792
3,58172_003247,58172,3247,43406,2020-09-13T19:30:20.200Z,-272,home,WR,14,33.238440,13.661136,1.655064,0.164592,330.50,344.03,0.566928,0.356616
4,58172_003247,58172,3247,48233,2020-09-13T19:30:20.200Z,-272,away,RB,45,69.585840,29.397960,3.072384,0.310896,166.59,180.56,0.804672,-0.795528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14867,58168_003392,58168,3392,43395,2020-09-11T03:01:57.300Z,92,away,FS,23,46.661832,12.463272,1.554480,0.155448,21.01,18.32,0.320040,-0.182880
14868,58168_003392,58168,3392,39947,2020-09-11T03:01:57.300Z,92,home,T,72,39.072312,16.934688,1.088136,0.118872,331.22,332.97,0.530352,-0.512064
14869,58168_003392,58168,3392,44822,2020-09-11T03:01:57.300Z,92,home,QB,15,32.013144,17.080992,1.353312,0.137160,165.22,162.66,0.347472,-0.064008
14870,58168_003392,58168,3392,38590,2020-09-11T03:01:57.300Z,92,home,G,70,39.977568,16.678656,1.408176,0.146304,341.08,336.00,0.374904,-0.374904


In [37]:
def convert_angular_direction(theta):
    angle = 90 - theta
    if angle < 0:
        angle = 360 + angle
    return angle

# 각도 변환
angle_columns = ['direction', 'orientation']
tr_player_tracking[angle_columns] = tr_player_tracking[angle_columns].applymap(convert_angular_direction)
te_player_tracking[angle_columns] = te_player_tracking[angle_columns].applymap(convert_angular_direction)

In [38]:
tr_player_tracking

,game_play,game_key,play_id,nfl_player_id,datetime,step,team,position,jersey_number,x_position,y_position,speed,distance,direction,orientation,acceleration,sa
0,58580_001136,58580,1136,44830,2021-10-10T21:08:20.900Z,-108,away,CB,22,56.317896,38.953440,1.014984,0.100584,129.67,186.07,0.649224,-0.585216
1,58580_001136,58580,1136,47800,2021-10-10T21:08:20.900Z,-108,away,DE,97,54.388512,24.515064,0.210312,0.009144,103.16,202.84,1.179576,0.822960
2,58580_001136,58580,1136,52444,2021-10-10T21:08:20.900Z,-108,away,FS,29,66.010536,28.767024,0.557784,0.054864,78.23,202.31,0.576072,-0.301752
3,58580_001136,58580,1136,46206,2021-10-10T21:08:20.900Z,-108,home,TE,86,52.459128,20.226528,0.338328,0.036576,322.15,26.37,0.630936,0.566928
4,58580_001136,58580,1136,52663,2021-10-10T21:08:20.900Z,-108,away,ILB,48,57.835800,25.146000,0.466344,0.045720,266.38,196.29,0.283464,0.283464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1353048,58575_003081,58575,3081,48476,2021-10-10T19:22:06.800Z,140,away,CB,40,66.092832,47.365920,1.453896,0.155448,104.64,107.32,0.448056,-0.374904
1353049,58575_003081,58575,3081,44887,2021-10-10T19:22:06.800Z,140,away,ILB,45,68.360544,49.834800,0.896112,0.091440,107.61,95.54,0.576072,-0.219456
1353050,58575_003081,58575,3081,44174,2021-10-10T19:22:06.800Z,140,away,ILB,49,68.159376,50.977800,1.024128,0.100584,97.21,100.80,0.576072,0.420624
1353051,58575_003081,58575,3081,45217,2021-10-10T19:22:06.800Z,140,away,TE,82,73.554336,44.595288,1.764792,0.182880,109.22,103.49,0.484632,-0.466344


In [31]:
def inplace(func):
    def wrapper(data, inplace):                           # 호출할 함수를 감싸는 함수
        if inplace:
            func(data, inplace)
        else:
            df = data.copy()
            func(df, inplace)
            return df
    return wrapper                           # wrapper 함수 반환

## contact id 만드는 함수
@inplace
def create_contact_id(df, inplace: bool=False):
    # Create contact ids
    df["contact_id"] = (
        df["game_play"]
        + "_"
        + df["step"].astype("str")
        + "_"
        + df["nfl_player_id_1"].astype("str")
        + "_"
        + df["nfl_player_id_2"].astype("str")
    )
    return df

## contact id 펼치는 함수
@inplace
def expand_contact_id(df, inplace: bool=False):
    """
    Splits out contact_id into seperate columns.
    """
    df["game_play"] = df["contact_id"].str[:12]
    df[['step', 'nfl_player_id_1', 'nfl_player_id_2']] = df.contact_id.str.split('_', expand=True).iloc[:, 2:]
    df["step"] = df["step"].astype("int")

    temp = df['contact']
    df.drop(columns=['contact'], inplace=True)
    df['contact'] = temp
    return df

In [20]:
df = tr_player_tracking
df.loc[df.game_play == '58575_003081'].loc[df.step == 0]

,game_play,game_key,play_id,nfl_player_id,datetime,step,team,position,jersey_number,x_position,y_position,speed,distance,direction,orientation,acceleration,sa
1349951,58575_003081,58575,3081,46192,2021-10-10T19:21:52.800Z,0,away,TE,81,87.197184,36.219384,0.502920,0.054864,119.22,245.36,0.941832,0.914400
1349952,58575_003081,58575,3081,53605,2021-10-10T19:21:52.800Z,0,home,ILB,52,40.517064,34.710624,1.371600,0.137160,96.61,103.11,1.700784,1.655064
1349953,58575_003081,58575,3081,44029,2021-10-10T19:21:52.800Z,0,home,WR,85,40.178736,5.934456,1.033272,0.100584,90.01,62.84,2.112264,2.084832
1349954,58575_003081,58575,3081,47890,2021-10-10T19:21:52.800Z,0,home,OLB,98,40.133016,12.865608,0.758952,0.064008,87.91,69.21,1.399032,1.362456
1349955,58575_003081,58575,3081,43534,2021-10-10T19:21:52.800Z,0,away,SS,42,55.248048,20.007072,0.246888,0.027432,81.82,300.22,0.438912,0.429768
1349956,58575_003081,58575,3081,42508,2021-10-10T19:21:52.800Z,0,home,P,8,40.343328,25.767792,4.526280,0.457200,109.83,141.29,1.709928,-1.042416
1349957,58575_003081,58575,3081,42837,2021-10-10T19:21:52.800Z,0,away,RB,34,86.968584,14.584680,1.746504,0.173736,9.70,289.64,0.667512,0.667512
1349958,58575_003081,58575,3081,46915,2021-10-10T19:21:52.800Z,0,home,TE,86,40.352472,42.711624,1.069848,0.128016,79.55,159.27,1.691640,1.673352
1349959,58575_003081,58575,3081,44174,2021-10-10T19:21:52.800Z,0,away,ILB,49,59.335416,37.664136,0.301752,0.027432,107.90,178.40,0.612648,0.612648
1349960,58575_003081,58575,3081,41356,2021-10-10T19:21:52.800Z,0,home,CB,29,40.160448,44.951904,0.438912,0.054864,92.41,96.23,1.280160,1.179576


In [13]:
N = 22
count = 0
for i in range(N):
    print(i, 'G')
    for j in range(i+1, N):
        print(i, j)
        count += 1

print(count)

0 G
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
1 G
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
1 16
1 17
1 18
1 19
1 20
1 21
2 G
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
2 14
2 15
2 16
2 17
2 18
2 19
2 20
2 21
3 G
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
3 12
3 13
3 14
3 15
3 16
3 17
3 18
3 19
3 20
3 21
4 G
4 5
4 6
4 7
4 8
4 9
4 10
4 11
4 12
4 13
4 14
4 15
4 16
4 17
4 18
4 19
4 20
4 21
5 G
5 6
5 7
5 8
5 9
5 10
5 11
5 12
5 13
5 14
5 15
5 16
5 17
5 18
5 19
5 20
5 21
6 G
6 7
6 8
6 9
6 10
6 11
6 12
6 13
6 14
6 15
6 16
6 17
6 18
6 19
6 20
6 21
7 G
7 8
7 9
7 10
7 11
7 12
7 13
7 14
7 15
7 16
7 17
7 18
7 19
7 20
7 21
8 G
8 9
8 10
8 11
8 12
8 13
8 14
8 15
8 16
8 17
8 18
8 19
8 20
8 21
9 G
9 10
9 11
9 12
9 13
9 14
9 15
9 16
9 17
9 18
9 19
9 20
9 21
10 G
10 11
10 12
10 13
10 14
10 15
10 16
10 17
10 18
10 19
10 20
10 21
11 G
11 12
11 13
11 14
11 15
11 16
11 17
11 18
11 19
11 20
11 21
12 G
12 13
12 14
12 15
12 16
12 17
12 18
12 

In [27]:
te_labels = submission.copy()
expand_contact_id(te_labels, inplace=True)
te_labels

,contact_id,game_play,step,nfl_player_id_1,nfl_player_id_2,contact
0,58168_003392_0_38590_43854,58168_003392,0,38590,43854,0
1,58168_003392_0_38590_41257,58168_003392,0,38590,41257,0
2,58168_003392_0_38590_41944,58168_003392,0,38590,41944,0
3,58168_003392_0_38590_42386,58168_003392,0,38590,42386,0
4,58168_003392_0_38590_47944,58168_003392,0,38590,47944,0
...,...,...,...,...,...,...
49583,58172_003247_125_40656_G,58172_003247,125,40656,G,0
49584,58172_003247_125_52521_G,58172_003247,125,52521,G,0
49585,58172_003247_125_52939_G,58172_003247,125,52939,G,0
49586,58172_003247_125_39008_G,58172_003247,125,39008,G,0


## 2. Data preprocessing

In [29]:
## 지면 충돌
tr_ground_contact = tr_labels[tr_labels.nfl_player_id_2 == 'G']
te_ground_contact = te_labels[te_labels.nfl_player_id_2 == 'G']

## 선수간 충돌
tr_players_contact = tr_labels[tr_labels.nfl_player_id_2 != 'G']
te_players_contact = te_labels[te_labels.nfl_player_id_2 != 'G']

tr_players_contact.nfl_player_id_2 = tr_players_contact.nfl_player_id_2.astype(int)
te_players_contact.nfl_player_id_2 = te_players_contact.nfl_player_id_2.astype(int)
tr_ground_contact

,contact_id,game_play,datetime,step,nfl_player_id_1,nfl_player_id_2,contact
231,58168_003392_0_38590_G,58168_003392,2020-09-11T03:01:48.100Z,0,38590,G,0
232,58168_003392_0_43854_G,58168_003392,2020-09-11T03:01:48.100Z,0,43854,G,0
233,58168_003392_0_41257_G,58168_003392,2020-09-11T03:01:48.100Z,0,41257,G,0
234,58168_003392_0_41944_G,58168_003392,2020-09-11T03:01:48.100Z,0,41944,G,0
235,58168_003392_0_42386_G,58168_003392,2020-09-11T03:01:48.100Z,0,42386,G,0
...,...,...,...,...,...,...,...
4721613,58582_003121_91_48220_G,58582_003121,2021-10-12T02:42:29.100Z,91,48220,G,0
4721614,58582_003121_91_47906_G,58582_003121,2021-10-12T02:42:29.100Z,91,47906,G,0
4721615,58582_003121_91_38557_G,58582_003121,2021-10-12T02:42:29.100Z,91,38557,G,0
4721616,58582_003121_91_47872_G,58582_003121,2021-10-12T02:42:29.100Z,91,47872,G,0


In [30]:
tr_players_contact

,contact_id,game_play,datetime,step,nfl_player_id_1,nfl_player_id_2,contact
0,58168_003392_0_38590_43854,58168_003392,2020-09-11T03:01:48.100Z,0,38590,43854,0
1,58168_003392_0_38590_41257,58168_003392,2020-09-11T03:01:48.100Z,0,38590,41257,0
2,58168_003392_0_38590_41944,58168_003392,2020-09-11T03:01:48.100Z,0,38590,41944,0
3,58168_003392_0_38590_42386,58168_003392,2020-09-11T03:01:48.100Z,0,38590,42386,0
4,58168_003392_0_38590_47944,58168_003392,2020-09-11T03:01:48.100Z,0,38590,47944,0
...,...,...,...,...,...,...,...
4721591,58582_003121_91_47872_52493,58582_003121,2021-10-12T02:42:29.100Z,91,47872,52493,0
4721592,58582_003121_91_47872_52500,58582_003121,2021-10-12T02:42:29.100Z,91,47872,52500,0
4721593,58582_003121_91_47872_48220,58582_003121,2021-10-12T02:42:29.100Z,91,47872,48220,0
4721594,58582_003121_91_47872_47906,58582_003121,2021-10-12T02:42:29.100Z,91,47872,47906,0


In [47]:
df = tr_players_contact.copy()

for game_play in tr_players_contact.game_play.unique():
    game_df = tr_players_contact[tr_players_contact.game_play == game_play]
    for step in game_df.step.unique():
        df = game_df[game_df.step == step]
        if len(df) != 231:
            print('ㅈ됐으요')
            print(len(df))
            print(f'game_play: {game_play}, step: {step}')

del game_df
del df

# msk1 = df.game_play == '58168_003392'
# msk2 = df.step == 12
# msk = pd.concat((msk1, msk2), axis=1)
# slct = msk.all(axis=1)

ㅈ됐으요
210
game_play: 58538_002774, step: 0
ㅈ됐으요
210
game_play: 58538_002774, step: 1
ㅈ됐으요
210
game_play: 58538_002774, step: 2
ㅈ됐으요
210
game_play: 58538_002774, step: 3
ㅈ됐으요
210
game_play: 58538_002774, step: 4
ㅈ됐으요
210
game_play: 58538_002774, step: 5
ㅈ됐으요
210
game_play: 58538_002774, step: 6
ㅈ됐으요
210
game_play: 58538_002774, step: 7
ㅈ됐으요
210
game_play: 58538_002774, step: 8
ㅈ됐으요
210
game_play: 58538_002774, step: 9
ㅈ됐으요
210
game_play: 58538_002774, step: 10
ㅈ됐으요
210
game_play: 58538_002774, step: 11
ㅈ됐으요
210
game_play: 58538_002774, step: 12
ㅈ됐으요
210
game_play: 58538_002774, step: 13
ㅈ됐으요
210
game_play: 58538_002774, step: 14
ㅈ됐으요
210
game_play: 58538_002774, step: 15
ㅈ됐으요
210
game_play: 58538_002774, step: 16
ㅈ됐으요
210
game_play: 58538_002774, step: 17
ㅈ됐으요
210
game_play: 58538_002774, step: 18
ㅈ됐으요
210
game_play: 58538_002774, step: 19
ㅈ됐으요
210
game_play: 58538_002774, step: 20
ㅈ됐으요
210
game_play: 58538_002774, step: 21
ㅈ됐으요
210
game_play: 58538_002774, step: 22
ㅈ됐으요
210
game_play: 5

### Physical Analysis

In [49]:
def acc_perpendicular(direct: float, acc: float, sa: float) -> float:
    res = np.sqrt(acc ** 2 - sa ** 2)
    if direct < 0:
        res = -res
    return res

def vectorization(data: pd.DataFrame) -> pd.DataFrame:
    deg2rad = np.pi / 180
    df = data.copy()

    df['x_vel'] = df.apply(lambda x: x.speed * np.cos(x.direction * deg2rad),
                           axis=1)
    df['y_vel'] = df.apply(lambda x: x.speed * np.sin(x.direction * deg2rad),
                           axis=1)

    df['sign_perpend'] = df.direction.diff().replace({0: np.nan})
    df['sign_perpend'] = df['sign_perpend'].fillna(method='ffill').fillna(method='bfill')
    df['sa_perpend'] = df.apply(lambda x: acc_perpendicular(x.sign_perpend, x.acceleration, x.sa),
                                axis=1)

    df['x_acc'] = df.apply(lambda x: x.sa * np.cos(x.direction * deg2rad), axis=1) + \
                  df.apply(lambda x: x.sa_perpend * np.cos((90 + x.direction) * deg2rad), axis=1)
    df['y_acc'] = df.apply(lambda x: x.sa * np.sin(x.direction * deg2rad), axis=1) + \
                  df.apply(lambda x: x.sa_perpend * np.sin((90 + x.direction) * deg2rad), axis=1)
    
    return df

In [50]:
tracking_ls = []
for game_play in tr_player_tracking.game_play.unique():
    game_df = tr_player_tracking[tr_player_tracking.game_play == game_play]
    temp_ls = []

    for player_id in game_df.nfl_player_id.unique():
        game_player_df = game_df[game_df.nfl_player_id == player_id]
        temp_ls.append(vectorization(game_player_df))

    temp_df = pd.concat(temp_ls, axis=0)
    temp_df = temp_df.sort_values(by='step', ascending=True)
    tracking_ls.append(temp_df)

train_player_tracking = pd.concat(tracking_ls, axis=0)
train_player_tracking.sort_index(inplace=True)

del temp_df
del tracking_ls
display(train_player_tracking)

,game_play,game_key,play_id,nfl_player_id,datetime,step,team,position,jersey_number,x_position,...,direction,orientation,acceleration,sa,x_vel,y_vel,sign_perpend,sa_perpend,x_acc,y_acc
0,58580_001136,58580,1136,44830,2021-10-10T21:08:20.900Z,-108,away,CB,22,56.317896,...,129.67,186.07,0.649224,-0.585216,-0.647930,0.781268,2.34,0.281094,0.157213,-0.629901
1,58580_001136,58580,1136,47800,2021-10-10T21:08:20.900Z,-108,away,DE,97,54.388512,...,103.16,202.84,1.179576,0.822960,-0.047882,0.204789,-19.94,-0.845066,0.635509,0.993745
2,58580_001136,58580,1136,52444,2021-10-10T21:08:20.900Z,-108,away,FS,29,66.010536,...,78.23,202.31,0.576072,-0.301752,0.113779,0.546056,3.40,0.490719,-0.541953,-0.195309
3,58580_001136,58580,1136,46206,2021-10-10T21:08:20.900Z,-108,home,TE,86,52.459128,...,322.15,26.37,0.630936,0.566928,0.267151,-0.207597,-3.82,-0.276899,0.277754,-0.566510
4,58580_001136,58580,1136,52663,2021-10-10T21:08:20.900Z,-108,away,ILB,48,57.835800,...,266.38,196.29,0.283464,0.283464,-0.029444,-0.465414,-0.44,-0.000000,-0.017898,-0.282898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1353048,58575_003081,58575,3081,48476,2021-10-10T19:22:06.800Z,140,away,CB,40,66.092832,...,104.64,107.32,0.448056,-0.374904,-0.367465,1.406692,3.32,0.245359,-0.142638,-0.424745
1353049,58575_003081,58575,3081,44887,2021-10-10T19:22:06.800Z,140,away,ILB,45,68.360544,...,107.61,95.54,0.576072,-0.219456,-0.271106,0.854118,-3.55,-0.532633,0.574066,-0.048031
1353050,58575_003081,58575,3081,44174,2021-10-10T19:22:06.800Z,140,away,ILB,49,68.159376,...,97.21,100.80,0.576072,0.420624,-0.128535,1.016030,3.41,0.393617,-0.443296,0.367897
1353051,58575_003081,58575,3081,45217,2021-10-10T19:22:06.800Z,140,away,TE,82,73.554336,...,109.22,103.49,0.484632,-0.466344,-0.580963,1.666425,0.30,0.131877,0.028993,-0.483764


In [26]:
tracking_ls = []
for game_play in te_player_tracking.game_play.unique():
    game_df = te_player_tracking[te_player_tracking.game_play == game_play]
    temp_ls = []

    for player_id in game_df.nfl_player_id.unique():
        game_player_df = game_df[game_df.nfl_player_id == player_id]
        temp_ls.append(vectorization(game_player_df))

    temp_df = pd.concat(temp_ls, axis=0)
    temp_df = temp_df.sort_values(by='step', ascending=True)
    tracking_ls.append(temp_df)

test_player_tracking = pd.concat(tracking_ls, axis=0)
test_player_tracking.sort_index(inplace=True)

del temp_df
del tracking_ls
display(test_player_tracking)

NameError: ignored

In [ ]:
train_player_tracking.to_csv(data_path / 'train_player_tracking_ver2.csv')
test_player_tracking.to_csv(data_path / 'test_player_tracking_ver2.csv')

In [62]:
def square(x):
    return x ** 2

def relative_quantative(data1: pd.Series, data2: pd.Series):
    # 동일 game_play, step에 대해 player1, player2에 대한 row of tracking data를 input으로 받는다.
    # position, velocitiy, acceleration
    # game_play, nfl_player_id, datetime, step
    # if isinstance(data1, pd.Series) and isinstance(data2, pd.Series):
    #     raise TypeError('Input data type must be Series.')
    # elif (data1.game_play != data2.game_play) or (data1.step != data2.step):
    #     raise ValueError('Time range between two dataframes does not match.')

    rel_df = data1[['game_play', 'step']]
    rel_df['nfl_player_id_1'] = data1.nfl_player_id
    rel_df['nfl_player_id_2'] = data2.nfl_player_id

    # 선수1과 선수2 사이의 변위, 상대속도, 상대가속도
    vector_quantative = ['x_position', 'y_position', 'x_vel', 'y_vel', 'x_acc', 'y_acc']
    # 선수1과 선수2 사이의 거리, 상대속력, 상대가속력
    scalar_quantative = ['dist', 'speed', 'acc']

    # 선수1과 선수2 사이의 변위, 상대속도, 상대가속도 계산
    for idx, q in enumerate(vector_quantative):
        rel_df[q] = data1[q] - data2[q]
        if idx % 2 == 1:
            prev_q = vector_quantative[idx//2]
            scalar = scalar_quantative[idx//2]
            rel_df[scalar] = rel_df[q].apply(square) + rel_df[prev_q].apply(square)
            rel_df[scalar] = rel_df[scalar].apply(np.sqrt)

    return rel_df

In [ ]:
for game_play in train_player_tracking.game_play.unique():
    game_df = train_player_tracking[train_player_tracking.game_play == game_play]
    temp_ls = []
    for step in game_df.step.unique():
        game_step_df = game_df[game_df.step == step]
        players = game_step_df.nfl_player_id.values

        for player1_idx in range(len(players)):
            player1_df = game_step_df[game_step_df.nfl_player_id == players[player1_idx]]
            for player2_idx in range(player1_idx + 1, len(players)):
                player2_df = game_step_df[game_step_df.nfl_player_id == players[player2_idx]]
                temp_ls.append(relative_quantative(player1_df, player2_df))

    temp_df = pd.concat(temp_ls, axis=0)
    del temp_ls
    temp_df.to_csv(data_path / 'refine' / 'train' / game_play + '_player_relative_tracking.csv')

In [ ]:
gp = train_player_tracking.game_play.unique()


In [ ]:
temp_df.to_csv(data_path / 'train_player_relative_tracking.csv')

In [ ]:
def make_graph_tracking(tracking):
    temp_ls = []
    for game_play in tracking.game_play.unique():
        game_df = tracking[tracking.game_play == game_play]

        for step in game_df.step.unique():
            game_step_df = game_df[game_df.step == step]
            players = game_step_df.nfl_player_id.values

            for player1_idx in range(len(players)):
                player1_df = game_step_df[game_step_df.nfl_player_id == players[player1_idx]]
                for player2_idx in range(player1_idx + 1, len(players)):
                    player2_df = game_step_df[game_step_df.nfl_player_id == players[player2_idx]]
                    temp_ls.append(relative_quantative(player1_df, player2_df))

    temp_df = pd.concat(temp_ls, axis=0)
    del temp_ls
    return temp_df

## Dask를 이용한 대용량 데이터처리

In [ ]:
!pip install dask